# Validate a Model's Environment Contains the Required Dependencies
This example demonstrates how to check that a model's environment contains all the dependencies it needs to run.
This allows developers and data scientists to validate the environment locally before attempting to deploy the
model to Verta.

## Imports

In [ ]:
import pandas as pd

from verta import Client
from verta.environment import Python
from verta.registry import VertaModelBase, verify_io, check_model_dependencies
from verta.registry.entities import RegisteredModel,  RegisteredModelVersion
from verta.utils import ModelAPI

## Verta Client Setup

In [ ]:
# Use local env vars or uncomment and fill out the lines below:
# os.environ['VERTA_EMAIL'] = ''
# os.environ['VERTA_DEV_KEY'] = ''
# os.environ['VERTA_HOST'] = ''

client = Client()

# Example Model Class

In [ ]:
class ExampleModel(VertaModelBase):
    def __init__(self, artifacts):
        pass

    @verify_io
    def predict(self, input):
        df = pd.DataFrame(input)
        return df.iloc[:1]

# Check Model Dependencies Manually

In [ ]:
assumed_environment = Python(requirements=["numpy"])  # incorrect package to support `pandas` dependency in model

# Raises a warning by default
check_model_dependencies(model_cls=ExampleModel, environment=assumed_environment)

# Raises an exception if `raise_for_missing=True`
check_model_dependencies(model_cls=ExampleModel, environment=assumed_environment, raise_for_missing=True)

# Check Dependencies While Creating a Registered Model Version

## Create a Registered Model

In [ ]:
registered_model: RegisteredModel = client.get_or_create_registered_model(
    name="pandas_dependent_model_example", labels=["examples"])

## Create a Registered Model Version
_Note the addition of the argument `check_model_dependencies=True` to trigger the dependency check._

This is the equivalent of running:
```python
verta.registry.check_model_dependencies(model_cls=ExampleModel, environment=Python([]), raise_for_missing=True)
```

In [ ]:
model_api = ModelAPI(
    inputs="dataframe",
    outputs="dataframe",
)

model_version: RegisteredModelVersion = registered_model.create_standard_model(
    model_cls=ExampleModel,
    environment=Python([]),
    model_api=model_api,
    check_model_dependencies=True
)